# 🤖 3. Ders Ekstrası : Keşifsel Veri Analizi (EDA) ve İstatistiksel Raporlama

Makine öğrenmesi modellerini eğitmeden önce veriyi anlamak, sınırlarını çizmek ve doğru öznitelikleri (features) seçmek, modelin nihai başarısı için atılması gereken en kritik adımdır. Bu bölümde, ham veri seti üzerinde **Keşifsel Veri Analizi (Exploratory Data Analysis - EDA)** sürecini uygulayacağız.

### Neden EDA Yapıyoruz?
Yapay zeka algoritmaları, kendilerine verilen verilerdeki matematiksel örüntüleri öğrenmek üzere tasarlanmıştır. Eğer veri setinin içinde tutarsızlıklar veya hedef çıktıyla hiçbir bağı bulunmayan "gürültülü" (noise) değişkenler varsa, model yanlış kurallar çıkaracaktır (Garbage In, Garbage Out). EDA, veriyi algoritmaya vermeden önce istatistiksel metotlarla denetleme ve temizleme sürecidir.

### Analiz Adımları ve İstatistiksel Yaklaşım

1. **İstatistiksel Özetleme:** Veri seti yüklendiğinde; standart sapma, aritmetik ortalama ve minimum/maksimum değerleri hesaplanır. Bu adım, verideki olası aykırı değerleri (outliers) ilk bakışta saptamak içindir.
2. **Dağılım Analizi (Histogram & KDE):** Değişkenlerin normal dağılıma uyup uymadığını ve verilerin hangi aralıklarda yığıldığını saptamak için kullanılır.
3. **Trend ve Regresyon Gözlemi (Scatter Plot):** İki değişken arasındaki ilişki haritalandırılır. Çekilen regresyon eğrisi (Curve Fitting), artış veya azalış trendlerini doğrusal olarak ispatlar.
4. **Korelasyon Isı Haritası (Correlation Heatmap):** Öznitelik Seçimi (Feature Selection) aşamasının temelini oluşturur. Değişkenlerin birbiriyle olan matematiksel bağı -1 (ters orantı) ile +1 (doğru orantı) arasında puanlanır.

### Dinamik Öznitelik Seçimi (Feature Selection)
Aşağıdaki kod bloğu yalnızca grafiksel bir döküm sunmakla kalmaz, aynı zamanda korelasyon matrisinin değerlerini çalışma zamanında (runtime) okuyarak dinamik bir mühendislik kararı üretir. Hedef değişken üzerinde kayda değer bir matematiksel etkisi olmayan girdilerin (örneğin yaş verisinin) eğitim setinden çıkarılması tavsiyesinde bulunarak, modelin ezberleme (overfitting) riskini düşürür ve hesaplama maliyetini azaltır.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Gereksiz uyarıları gizliyoruz
warnings.filterwarnings('ignore')

print("-" * 75)
print("KEŞİFSEL VERİ ANALİZİ (EDA) VE İSTATİSTİKSEL RAPORLAMA")
print("-" * 75)

# --- 1. ADIM: SENTETİK BÜYÜK VERİ SETİ OLUŞTURMA ---
np.random.seed(42)
veri_sayisi = 1000

yas = np.random.normal(35, 10, veri_sayisi).astype(int) 
gelir = np.random.normal(50000, 15000, veri_sayisi)     
sitede_gecirilen_sure = np.random.normal(15, 5, veri_sayisi) 

# Harcama puanını hesaplayan formül (Korelasyon yaratmak için)
harcama_puani = (gelir / 1000) + (sitede_gecirilen_sure * 2) + np.random.normal(0, 10, veri_sayisi)

df = pd.DataFrame({
    'Yas': yas,
    'Yillik_Gelir_TL': gelir,
    'Sitede_Gecirilen_Sure_Dk': sitede_gecirilen_sure,
    'Harcama_Puani': harcama_puani
})

print("BİLGİ: 1000 satırlık müşteri veri seti başarıyla oluşturuldu.\n")

# İSTATİSTİKSEL ÖZET TABLOSU
print("VERİ SETİ İSTATİSTİKSEL ÖZETİ (df.describe):")
print("-" * 75)
print(df.describe().round(2))
print("-" * 75)

# --- 2. ADIM: VERİ GÖRSELLEŞTİRME ---
plt.style.use('bmh') 
fig = plt.figure(figsize=(18, 5))
fig.canvas.manager.set_window_title("Keşifsel Veri Analizi (EDA)")

# Grafik 1: Yaş Dağılımı (Histogram)
ax1 = plt.subplot(1, 3, 1)
sns.histplot(df['Yas'], bins=30, kde=True, color='steelblue', ax=ax1)
ax1.set_title("Müşteri Yaş Dağılımı", fontweight='bold')
ax1.set_xlabel("Yaş")
ax1.set_ylabel("Kişi Sayısı")

# Grafik 2: Gelir - Harcama Regresyonu (Scatter Plot)
ax2 = plt.subplot(1, 3, 2)
sns.regplot(x='Yillik_Gelir_TL', y='Harcama_Puani', data=df, 
            scatter_kws={'alpha':0.3, 'color':'darkorange', 's':20}, 
            line_kws={'color':'red', 'linewidth':2}, ax=ax2)
ax2.set_title("Gelir ve Harcama Puanı İlişkisi", fontweight='bold')
ax2.set_xlabel("Yıllık Gelir (TL)")
ax2.set_ylabel("Harcama Puanı")

# Grafik 3: Isı Haritası (Correlation Heatmap)
ax3 = plt.subplot(1, 3, 3)
korelasyon_matrisi = df.corr()
sns.heatmap(korelasyon_matrisi, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, ax=ax3)
ax3.set_title("Değişkenler Arası Korelasyon Haritası", fontweight='bold')

plt.tight_layout()
print("\nBİLGİ: Grafikler ekrana yansıtılıyor. İleri düzey analiz raporu, grafiği kapattıktan sonra oluşturulacaktır...")
plt.show()

# --- 3. ADIM: DİNAMİK İSTATİSTİK VE KORELASYON ANALİZİ ---
kor_gelir_harcama = korelasyon_matrisi.loc['Yillik_Gelir_TL', 'Harcama_Puani']
kor_sure_harcama = korelasyon_matrisi.loc['Sitede_Gecirilen_Sure_Dk', 'Harcama_Puani']
kor_yas_harcama = korelasyon_matrisi.loc['Yas', 'Harcama_Puani']

ortalama_gelir = df['Yillik_Gelir_TL'].mean()
ortalama_yas = df['Yas'].mean()
ortalama_sure = df['Sitede_Gecirilen_Sure_Dk'].mean()

print("\n" + "=" * 75)
print("DİNAMİK VERİ ANALİZİ VE MÜHENDİSLİK RAPORU")
print("=" * 75)

print("\n1. TEMEL İSTATİSTİKSEL BULGULAR:")
print(f"   -> Müşterilerin yaş ortalaması: {ortalama_yas:.0f}")
print(f"   -> Yıllık ortalama gelir düzeyi: {ortalama_gelir:.2f} TL")
print(f"   -> Sitede geçirilen ortalama süre: {ortalama_sure:.1f} Dakika")

print("\n2. KORELASYON (İLİŞKİ) ANALİZİ:")
print(f"   -> Yıllık Gelir ve Harcama Puanı Korelasyonu         : {kor_gelir_harcama:.2f} (Çok Güçlü Pozitif İlişki)")
print(f"   -> Sitede Geçirilen Süre ve Harcama Puanı Korelasyonu: {kor_sure_harcama:.2f} (Orta/Güçlü Pozitif İlişki)")
print(f"   -> Yaş ve Harcama Puanı Korelasyonu                  : {kor_yas_harcama:.2f} (İlişki Yok)")

print("\n3. MAKİNE ÖĞRENMESİ (FEATURE SELECTION) KARARI:")
if kor_gelir_harcama > 0.60 and kor_sure_harcama > 0.40:
    print("   -> KARAR: Makine öğrenmesi modeli eğitilirken 'Yillik_Gelir_TL' ve")
    print("      'Sitede_Gecirilen_Sure_Dk' öznitelikleri ana girdiler (X) olarak kullanılacaktır.")
    print(f"   -> GEREKÇE: Her iki değişken de hedeflenen 'Harcama Puanı' (Y) üzerinde")
    print("      matematiksel olarak anlamlı ve pozitif bir etkiye sahiptir.")
    print(f"   -> İPTAL: 'Yas' değişkeni ({kor_yas_harcama:.2f}) harcama eğilimini etkilemediği için")
    print("      modeli yormamak adına veri setinden (Drop) çıkarılmalıdır.")
else:
    print("   -> KARAR: Veri setindeki değişkenler arasında hedefi tahmin etmeye yetecek")
    print("      kadar güçlü bir matematiksel bağ (korelasyon) bulunamamıştır.")
print("=" * 75 + "\n")